# Criando o caos
Aplicando a engenharia do caos para surpreender suas aplicações de missão crítica antes que elas o façam.

Provavelmente você já ouviu falar sobre engenharia do caos, conceito criado no Netflix e envolve quebrar coisas em produção de propósito com objetivo de certificar que a alta disponibilidade e a tolerância à falhas das suas aplicações funcionam.

Faremos aqui uma breve introdução, mas você pode conhecer mais sobre engenharia do caos no livro [Chaos Engineering da O'Reilly](https://www.oreilly.com/library/view/chaos-engineering/9781491988459/). Enquanto escrevemos esse curso você pode obtê-lo gratuitamente em [Verica - The Chaos Engineering Book](https://www.verica.io/the-chaos-engineering-book/).

> _Chaos Engineering é a disciplina de realizar experimentos sobre sistemas distribuídos com o intuito de construir confiança com relação a capacidade de um sistema distribuído suportar condições adversas em produção._ - [PRINCÍPIOS DE CHAOS ENGINEERING](https://principlesofchaos.org/pt/)

Estes experimentos seguem quatro etapas:

* **Comece definindo o que significa “sistema estável”**, uma medida que tem como resultado mensurável um indicativo sobre o comportamento normal do sistema.
* **Crie hipóteses** se o estado “sistema estável” permanecerá tanto no grupo de controle quanto no grupo onde o experimento será executado.
* **Introduza variáveis que reflitam eventos que ocorrem no mundo real**, como por exemplo: servidores que travam, discos rígidos defeituosos, conexões de rede que são interrompidas, etc.
* **Tente refutar cada hipótese** procurando diferenças entre o “sistema estável”, o grupo de controle e o grupo experimental.

Com kubernetes você tem uma série de controles e o Istio adiciona ainda mais variáveis que permitem, com um mínimo esforço, conduzir esses experimentos.

Veremos como implementar a injeção de falhas e lentidão do tempo de resposta para o nosso experimento.

## Injetando falhas e atrasos

TODO